In [1]:
--- Libraries:
arguments = require 'Settings.arguments'
constants = require 'Settings.constants'
card_to_string = require 'Game.card_to_string_conversion'
card_tools = require 'Game.card_tools'
game_settings = require 'Settings.game_settings'
Plot = require 'itorch.Plot'
nninit = require 'nninit'
require 'torch'
require 'math'
require 'Tree.tree_builder'
require 'Tree.tree_data_generation'
require 'Tree.tree_visualiser'
require 'nn'
require 'Tree.tree_cfr'
require 'nngraph'
require 'optim'
require 'image'
require 'NN.strategy_net_builder'
require 'NN.strategy_net_trainer'



In [2]:
--- Create the tree
builder = PokerTreeBuilder()
--- Parameters for the tree
params = {}
params.root_node = {}
params.root_node.board = card_to_string:string_to_board('')
params.root_node.street = 1
params.root_node.current_player = constants.players.P1
params.root_node.bets = arguments.Tensor{200, 200}

--- BUild tree
tree = builder:build_tree(params)
--- build data
game = TreeData(tree)


In [3]:
--- CFR Solver
local starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
starting_ranges[1]:copy(card_tools:get_uniform_range(params.root_node.board))
starting_ranges[2]:copy(card_tools:get_uniform_range(params.root_node.board))
local tree_cfr = TreeCFR()
print("Solver")
tree_cfr:run_cfr(tree, starting_ranges)
print("geting training set")
game:get_training_set(tree,1)

Solver


geting training set


## Using the modules

In [4]:
strategy_nn = StrategyNN()
nn_model = strategy_nn.model

In [5]:
-- Building the neural net model
strategy_nn = StrategyNN()
nn_model = strategy_nn.model

-- Building trainer:
nn_trainer = NNTrainer(game,nn_model)

-- Criterion definition:
---criterion = nn.SmoothL1Criterion()

train_data = nn_trainer.train_data

---loss_x = criterion:forward(nn_model:forward({train_data.features,train_data.masks}), train_data.targets)
---print(loss_x)
---print(nn_model:getParameters())

-- Options:
opt = {}
opt.learningRate = 0.001
opt.momentum = 0.95
opt.validate = true

In [6]:
number_ok = {}
train_loss_table = {}

test_loss_table = {}

local current_criterion = nn.BCECriterion()

for i = 1,1 do
    --- Trying a new model: 
    nn_model2 = nn_trainer:set_new_model()

    nn_trainer:generate_training_and_validation_sets()

    train_loss2,test_loss2= nn_trainer:train(nn_model2,current_criterion,opt,10000)

    ---train_loss,test_loss = train_and_test(data_train,data_test,mlp,criterion,opt)
    table.insert(train_loss_table, train_loss2)
    table.insert(test_loss_table, test_loss2)


    if torch.sum(train_loss2) < torch.sum(test_loss2) then
        table.insert(number_ok,1)
    end
end
print(torch.sum((torch.Tensor(number_ok))))

1


In [7]:
function print_avg_losses(train_loss_table,test_loss_table)
    n_samples = #train_loss_table

    local last_loss_train = torch.Tensor(n_samples):fill(0)
    local last_loss_test = torch.Tensor(n_samples):fill(0)
    for i = 1,n_samples do        
        last_loss_train[i] = train_loss_table[i][-1]
        last_loss_test[i] = test_loss_table[i][-1]
    end
    print(torch.mean(last_loss_train),torch.mean(last_loss_test))
---    print(torch.mean(starting_loss_test),torch.mean(last_loss_test))
end

In [8]:
print_avg_losses(train_loss_table,test_loss_table)

0.41491669416428	1.2776231765747


In [9]:
torch.std(train_loss_table[1])

0.011793768956922


In [11]:
local i = 1

t2 = torch.range(1,train_loss_table[i]:size(1))
local plot = Plot()
plot:line(t2,train_loss_table[i],'red' ,'train loss')
plot:line(t2,test_loss_table[i],'blue' ,'test loss')
plot:legend(true):title('Mean Square Error')
plot:draw()

In [10]:
--- Training NN
function train_and_test_minibatch(data_train,data_test,model,criterion,opt)
    local epochs = 1000
---    torch.manualSeed(1)
    --- Defining the parameters and the gradient
    local params, gradParameters = model:getParameters()
    local training_loss_tensor = {}
    local test_loss_tensor = {}
    --- Defining the function that gives back the loss and the gradient
    local train_loss = 0
    local test_loss = 0
    feval = function(params)
        --- Features:
        gradParameters:zero()
        
        -- Forward pass:
        model:forward({data_train.features,data_train.masks})
        local predictions = model.output

        -- Errors: 
        local loss = criterion:forward(predictions, data_train.targets)
        --Backprop:
        local gradCriterion = criterion:backward(predictions, data_train.targets)
        model:backward({data_train.features,data_train.masks}, gradCriterion)

        return loss,gradParameters
    end
    
    -- Perform SGD step:
    sgdState = sgdState or {
    learningRate = opt.learningRate,
    momentum = opt.momentum or 0,
    nesterov = opt.nesteov or false,
    learningRateDecay = 5e-7}
    
    adamState = adamState or {
    learningRate = opt.learningRate or 0.001,
    learningRateDecay = opt.learningRateDecay or 0.9,
    weightDecay = opt.weightDecay or 0.999}

    for i = 1,10000 do
        optim.sgd(feval, params, sgdState)

        -- Training loss:
        train_loss = criterion:forward(model:forward({data_train.features,data_train.masks}), data_train.targets)
        table.insert(training_loss_tensor,train_loss)

        -- Test loss:
        test_loss = criterion:forward(model:forward({data_test.features,data_test.masks}), data_test.targets)
        table.insert(test_loss_tensor,test_loss)
    end
    return torch.Tensor(training_loss_tensor),torch.Tensor(test_loss_tensor)
end

In [11]:
local StrategyCriterion, parent = torch.class('nn.StrategyCriterion', 'nn.Criterion')

function StrategyCriterion:__init(sizeAverage)
   parent.__init(self)
   if sizeAverage ~= nil then
     self.sizeAverage = sizeAverage
   else
     self.sizeAverage = true
   end
end

function StrategyCriterion:updateOutput(input, target)
   self.output_tensor = self.output_tensor or input.new(1)
   input.THNN.MSECriterion_updateOutput(
      input:cdata(),
      target:cdata(),
      self.output_tensor:cdata(),
      self.sizeAverage
   )
   self.output = self.output_tensor[1]
   return self.output
end

function StrategyCriterion:updateGradInput(input, target)
   input.THNN.MSECriterion_updateGradInput(
      input:cdata(),
      target:cdata(),
      self.gradInput:cdata(),
      self.sizeAverage
   )
   return self.gradInput
end